# PDR Assessor Data Analysis

In [1]:
#SF Open Data included very incomplete building SF data, so this cleans and analyzes City/County Assessor property tax data

In [1]:
import pandas as pd
import requests
import geopandas as gpd
import matplotlib.pyplot as plt
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
pip install xlrd

You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Accessing 2017-2018 property tax data
url = 'https://www.sfassessor.org/sites/default/files/uploaded/2019.8.12__SF_ASR_Secured_Roll_Data_2017-2018_0.xlsx'

import requests  
r = requests.get(url)  
open('AssessorPropertyData.xls', 'wb').write(r.content)  

32639786

In [4]:
sheet = pd.read_excel('AssessorPropertyData.xls')

In [5]:
pd.set_option("display.max.columns", None)
sheet.head()

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0


In [6]:
#Looking at the data

sheet.sample(n=10)

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL
140155,0000 0601 ALABAMA ST0109,09C,4021 014,26,Z,2009,2,3,0,0,0,NaN,NaN,0.00,0.0,1114,1114,0.0,NaN,NaN,1000.0,1.00000,11,NaN,0,7000,17,90910,0,327545,327545,0
91481,0000 2450 20TH AV0000,02D,2406 026,17,D,1929,1,0,6,1,1,RH1D,D,20.00,0.0,1415,0,3999.6,R,NaN,1000.0,1.00000,11,NaN,0,7000,17,890307,0,246575,404230,0
126722,0000 0199 NEW MONTGOMERY ST1508,09B,3722 242,25,Z,2004,0,0,0,0,0,NaN,NaN,0.00,0.0,653,0,0.0,NaN,NaN,1000.0,0.40000,NaN,NaN,0,0,17,110713,0,215221,297215,0
88929,0000 2397 21ST AV0000,02D,2350 017,17,C,1933,4,0,6,2,2,NC2,D,0.00,0.0,4225,0,3249.0,O,NaN,1000.0,0.50000,NaN,NaN,0,0,17,70921,0,762808,1144218,0
201071,0000 2973 21ST AV0000,03F,7205 005A,44,D,1931,3,4,10,1,1,RH1D,D,0.00,0.0,2432,742,4098.0,NaN,NaN,1000.0,1.00000,11,NaN,0,7000,17,0,0,144496,46354,0
46036,0000 0628 CENTRAL AV0000,06F,1185 021,9,A5,1906,6,0,24,3,6,RM1,D,6.25,0.0,4950,0,2762.5,R,NaN,1000.0,0.50000,NaN,NaN,0,0,17,910207,0,289436,253250,0
197194,0000 0809 SARGENT ST0000,03G,7083 027,42,D,1940,1,0,4,1,1,RH1,D,0.00,0.0,1000,0,2500.0,R,NaN,1000.0,1.00000,NaN,NaN,0,0,17,20110,0,183291,274937,0
84000,0000 2011 19TH AV0000,02D,2139 001A,16,F,1915,2,0,7,1,2,RH2,D,0.00,0.0,1292,0,2996.0,O,NaN,1000.0,0.33333,NaN,NaN,0,0,17,110408,0,209712,220746,0
127519,0000 0725 TEHAMA STIT 2,09F,3729 122,25,LZ,2000,2,0,0,0,1,NaN,NaN,0.00,0.0,1390,0,0.0,NaN,NaN,1000.0,1.00000,11,NaN,0,7000,17,100415,0,284723,425976,0
186928,3656 3654 MISSION ST0000,09G,6687 019,40,F,1965,4,0,12,2,2,RH2,D,0.00,0.0,3220,0,2487.0,NaN,NaN,1000.0,1.00000,NaN,NaN,0,0,17,0,0,291846,179245,0


In [7]:
#Figuring out how to merge RP1PRCLID, the parcel number, into a single 7 digit identifier
sheet.RP1PRCLID.describe()

count        208041
unique       208041
top       6596 013J
freq              1
Name: RP1PRCLID, dtype: object

In [8]:
sheet['parcelno'] = sheet['RP1PRCLID']

In [9]:
sheet.head()

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0001 001
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0002 001
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0004 002
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0005 001
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.0,NaN,NaN,1000.0,1.0,NaN,N,0,0,17,0,0,0,0,0,0006 001


In [10]:
sheet[['Block','Lot']] = sheet.parcelno.str.split(" ",expand=True,)
sheet

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno,Block,Lot
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0001 001,0001,001
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0002 001,0002,001
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0004 002,0004,002
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0005 001,0005,001
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0006 001,0006,001
5,0000 0000 0000,08H,0007 001,1,G,1900,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,113437.50,R,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0007 001,0007,001
6,0000 2801 LEAVENWORTH ST0000,08H,0010 001,1,C1,1907,0,0,0,3,0,C2,C,0.0,0.0,133675,0,66124.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,110523,0,9087359,13631039,0,0010 001,0010,001
7,0495 0455 JEFFERSON ST0000,08H,0010 002,1,H,1907,0,0,252,4,0,C2,NaN,0.0,0.0,203840,0,47131.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,0,0,0,1,0,0010 002,0010,002
8,0000 2800 LEAVENWORTH ST0000,08H,0011 007,1,HC,0,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,55687.50,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,947696,32668516,32668516,67032,0011 007,0011,007
9,0000 0500 BEACH ST0000,08H,0011 008,1,C1,1974,7,0,36,4,29,C2,A,75.0,0.0,198525,0,57750.00,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,0,16694570,32668516,0,0011 008,0011,008


In [11]:
sheet["parcelno2"] = sheet["Block"] + sheet["Lot"]

In [12]:
sheet

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno,Block,Lot,parcelno2
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0001 001,0001,001,0001001
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0002 001,0002,001,0002001
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0004 002,0004,002,0004002
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0005 001,0005,001,0005001
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0006 001,0006,001,0006001
5,0000 0000 0000,08H,0007 001,1,G,1900,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,113437.50,R,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0007 001,0007,001,0007001
6,0000 2801 LEAVENWORTH ST0000,08H,0010 001,1,C1,1907,0,0,0,3,0,C2,C,0.0,0.0,133675,0,66124.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,110523,0,9087359,13631039,0,0010 001,0010,001,0010001
7,0495 0455 JEFFERSON ST0000,08H,0010 002,1,H,1907,0,0,252,4,0,C2,NaN,0.0,0.0,203840,0,47131.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,0,0,0,1,0,0010 002,0010,002,0010002
8,0000 2800 LEAVENWORTH ST0000,08H,0011 007,1,HC,0,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,55687.50,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,947696,32668516,32668516,67032,0011 007,0011,007,0011007
9,0000 0500 BEACH ST0000,08H,0011 008,1,C1,1974,7,0,36,4,29,C2,A,75.0,0.0,198525,0,57750.00,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,0,16694570,32668516,0,0011 008,0011,008,0011008


In [13]:
#Cleaning up column names

sheet['zoning'] = sheet['ZONE']
sheet['buildingsf'] = sheet['SQFT']
sheet['landarea'] = sheet['LAREA']
sheet['landvalue'] = sheet['RP1LNDVAL']

In [14]:
sheet

,PROPLOC,RP1NBRCDE,RP1PRCLID,RP1VOLUME,RP1CLACDE,YRBLT,BATHS,BEDS,ROOMS,STOREYNO,UNITS,ZONE,CONSTTYPE,DEPTH,FRONT,SQFT,FBA,LAREA,LOTCODE,REPRISDATE,RP1TRACDE,OWNRPRCNT,EXEMPTYPE,RP1STACDE,RP1EXMVL2,RP1EXMVL1,ROLLYEAR,RECURRSALD,RP1FXTVAL,RP1IMPVAL,RP1LNDVAL,RP1PPTVAL,parcelno,Block,Lot,parcelno2,zoning,buildingsf,landarea,landvalue
0,0000 0000 0000,08H,0001 001,1,G,1900,0,0,0,0,0,P,NaN,0.0,0.0,0,0,27965.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0001 001,0001,001,0001001,P,0,27965.00,0
1,0000 0000 0000,08H,0002 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,8359.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0002 001,0002,001,0002001,C2,0,8359.00,0
2,0000 0000 0000,08H,0004 002,1,VCI,0,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,0.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0004 002,0004,002,0004002,C2,0,0.00,0
3,0000 0286 JEFFERSON ST0000,08H,0005 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,68153.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0005 001,0005,001,0005001,C2,0,68153.00,0
4,0000 0000 0000,08H,0006 001,1,G,1900,0,0,0,0,0,C2,NaN,0.0,0.0,0,0,108015.00,NaN,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0006 001,0006,001,0006001,C2,0,108015.00,0
5,0000 0000 0000,08H,0007 001,1,G,1900,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,113437.50,R,NaN,1000.0,1.0000,NaN,N,0,0,17,0,0,0,0,0,0007 001,0007,001,0007001,C2,0,113437.50,0
6,0000 2801 LEAVENWORTH ST0000,08H,0010 001,1,C1,1907,0,0,0,3,0,C2,C,0.0,0.0,133675,0,66124.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,110523,0,9087359,13631039,0,0010 001,0010,001,0010001,C2,133675,66124.00,13631039
7,0495 0455 JEFFERSON ST0000,08H,0010 002,1,H,1907,0,0,252,4,0,C2,NaN,0.0,0.0,203840,0,47131.00,NaN,NaN,1000.0,1.0000,NaN,NaN,0,0,17,0,0,0,1,0,0010 002,0010,002,0010002,C2,203840,47131.00,1
8,0000 2800 LEAVENWORTH ST0000,08H,0011 007,1,HC,0,0,0,0,0,0,C2,NaN,75.0,0.0,0,0,55687.50,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,947696,32668516,32668516,67032,0011 007,0011,007,0011007,C2,0,55687.50,32668516
9,0000 0500 BEACH ST0000,08H,0011 008,1,C1,1974,7,0,36,4,29,C2,A,75.0,0.0,198525,0,57750.00,R,NaN,1000.0,1.0000,NaN,NaN,0,0,17,41001,0,16694570,32668516,0,0011 008,0011,008,0011008,C2,198525,57750.00,32668516


In [15]:
#creating a new and clean data frame to work with
reldata = sheet[['parcelno2', 'zoning', 'buildingsf', 'landarea', 'landvalue']].copy()

In [16]:
reldata

,parcelno2,zoning,buildingsf,landarea,landvalue
0,0001001,P,0,27965.00,0
1,0002001,C2,0,8359.00,0
2,0004002,C2,0,0.00,0
3,0005001,C2,0,68153.00,0
4,0006001,C2,0,108015.00,0
5,0007001,C2,0,113437.50,0
6,0010001,C2,133675,66124.00,13631039
7,0010002,C2,203840,47131.00,1
8,0011007,C2,0,55687.50,32668516
9,0011008,C2,198525,57750.00,32668516


In [17]:
reldata.to_csv(r'AssessorDataNew.csv')

In [ ]:
#bringing existing parcel data set to merge assessor data with

In [19]:
parcelzoning = pd.read_csv('parcelwzoning.csv', low_memory=False)

In [20]:
parcelzoning.sample(n=8)

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning
35615,35615,35591,1773004,4000,0,RESIDENT,0,2975.867235,RH-2,True,MULTIPOLYGON (((-122.47608172488529 37.7622626...,RH-2
19522,19522,19494,1266010,3882,0,RESIDENT,0,3410.403315,RH-3,True,MULTIPOLYGON (((-122.45334762405818 37.7654635...,RH-3
53411,53411,53387,2174007,995,0,RESIDENT,0,2989.093346,RH-1,True,MULTIPOLYGON (((-122.50187864888127 37.7469166...,RH-1
50448,50448,50424,2090044,1162,0,RESIDENT,0,2953.274290,RH-1,True,MULTIPOLYGON (((-122.50362050119898 37.7504442...,RH-1
145965,145965,145957,5428F018,1626,0,RESIDENT,0,1732.487495,RH-1,True,MULTIPOLYGON (((-122.3978337102544 37.72849904...,RH-1
2485,2485,2480,0162001,9666,0,VISITOR,0,3208.972298,NCD,True,MULTIPOLYGON (((-122.40567945820216 37.7976621...,NCD
143811,143811,143809,7275032,2157,0,RESIDENT,0,3113.692355,RH-1(D),True,MULTIPOLYGON (((-122.49469657676617 37.7315123...,RH-1(D)
53358,53358,53334,2186042,2979,0,RESIDENT,0,2975.843301,RH-1,True,MULTIPOLYGON (((-122.48942334882054 37.7473302...,RH-1


In [21]:
parcelzoning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155468 entries, 0 to 155467
Data columns (total 12 columns):
Unnamed: 0     155468 non-null int64
objectid       155468 non-null int64
blklot         155468 non-null object
bldgsf         155468 non-null int64
totuses        155468 non-null int64
landuse        155468 non-null object
pdr            155468 non-null int64
parcelarea     155468 non-null float64
zoning_code    154673 non-null object
active         155451 non-null object
geometry       155468 non-null object
zoning         154673 non-null object
dtypes: float64(1), int64(5), object(6)
memory usage: 14.2+ MB


In [ ]:
#merging data, tried two different ways to see what would preserve the most non-null data

In [22]:
combined_parcel = pd.merge(left=parcelzoning, right=reldata, how='left', left_on='blklot', right_on='parcelno2')

In [23]:
combined_parcel.sample(n=8)

,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_x,parcelno2,zoning_y,buildingsf,landarea,landvalue
73118,73118,73098,3275A008,3125,0,RESIDENT,0,3466.940790,RH-1(D),True,MULTIPOLYGON (((-122.46141341387138 37.7290460...,RH-1(D),NaN,NaN,NaN,NaN,NaN
23264,23264,23237,1451035,1650,0,RESIDENT,0,2983.832062,RM-1,True,MULTIPOLYGON (((-122.47891677264282 37.7814226...,RM-1,1451035,RM1,1650.0,3000.0,271978.0
152639,152639,152637,0219012,21040,2411,MIXRES,0,6679.182372,RM-3,True,MULTIPOLYGON (((-122.41724362631888 37.7920345...,RM-3,0219012,RM3,21040.0,6664.0,1200602.0
77810,77810,77791,2998041,1698,0,RESIDENT,0,2575.799906,RH-1,True,MULTIPOLYGON (((-122.45134594037378 37.7369104...,RH-1,2998041,RH1,1698.0,2286.0,557758.0
119106,119106,119094,7238A011,1651,0,RESIDENT,0,3928.422381,RH-1(D),True,MULTIPOLYGON (((-122.47455346993584 37.7219348...,RH-1(D),NaN,NaN,NaN,NaN,NaN
85123,85123,85109,3584089,0,0,MIXRES,0,2812.840341,RH-3,True,MULTIPOLYGON (((-122.4312988497194 37.76071132...,RH-3,3584089,RH3,0.0,2848.0,39015.0
22714,22714,22687,1354020,3250,0,RESIDENT,0,4536.699971,RH-1,True,MULTIPOLYGON (((-122.46320104055063 37.7883862...,RH-1,1354020,RH1,4297.0,4560.0,2374185.0
80028,80028,80012,3624037,1795,0,RESIDENT,0,2815.777619,RH-2,True,MULTIPOLYGON (((-122.43334334451858 37.7533476...,RH-2,3624037,RH2,1795.0,2848.0,38988.0


In [24]:
combined_parcel.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155468 entries, 0 to 155467
Data columns (total 17 columns):
Unnamed: 0     155468 non-null int64
objectid       155468 non-null int64
blklot         155468 non-null object
bldgsf         155468 non-null int64
totuses        155468 non-null int64
landuse        155468 non-null object
pdr            155468 non-null int64
parcelarea     155468 non-null float64
zoning_code    154673 non-null object
active         155451 non-null object
geometry       155468 non-null object
zoning_x       154673 non-null object
parcelno2      145546 non-null object
zoning_y       140671 non-null object
buildingsf     145546 non-null float64
landarea       145546 non-null float64
landvalue      145546 non-null float64
dtypes: float64(4), int64(5), object(8)
memory usage: 21.4+ MB


In [ ]:
#second merged data variable

In [25]:
combined_assessor = pd.merge(left=reldata, right=parcelzoning, how='left', left_on='parcelno2', right_on='blklot')

In [26]:
combined_assessor.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208041 entries, 0 to 208040
Data columns (total 17 columns):
parcelno2      190328 non-null object
zoning_x       172651 non-null object
buildingsf     208041 non-null int64
landarea       208041 non-null float64
landvalue      208041 non-null int64
Unnamed: 0     145546 non-null float64
objectid       145546 non-null float64
blklot         145546 non-null object
bldgsf         145546 non-null float64
totuses        145546 non-null float64
landuse        145546 non-null object
pdr            145546 non-null float64
parcelarea     145546 non-null float64
zoning_code    144769 non-null object
active         145546 non-null object
geometry       145546 non-null object
zoning_y       144769 non-null object
dtypes: float64(7), int64(2), object(8)
memory usage: 28.6+ MB


In [27]:
combined_assessor.sample(n=8)

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y
91441,2405020,RH1D,1318,3998.0,384157,152800.0,152800.0,2405020,1318.0,0.0,RESIDENT,0.0,3974.421982,RH-1(D),True,MULTIPOLYGON (((-122.47718234013138 37.7413158...,RH-1(D)
72973,1834020,RH2,1260,2374.0,76623,38870.0,38846.0,1834020,1260.0,0.0,RESIDENT,0.0,2284.917458,RH-2,True,MULTIPOLYGON (((-122.47750576586985 37.7601249...,RH-2
4027,0080156,C2,941,0.0,137576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102975,2835008,RH1,1062,4216.0,693209,60900.0,60876.0,2835008,1062.0,0.0,RESIDENT,0.0,3601.501690,RH-1(D),True,MULTIPOLYGON (((-122.45322848316381 37.7502463...,RH-1(D)
177679,6410039,NaN,827,0.0,263670,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154346,5340023,RH1,1275,2495.0,17689,91366.0,91362.0,5340023,1275.0,0.0,RESIDENT,0.0,2479.176930,RH-1,True,MULTIPOLYGON (((-122.38832495989662 37.7314299...,RH-1
68959,1746006,RH2,4564,2375.0,36405,43602.0,43578.0,1746006,4564.0,0.0,RESIDENT,0.0,2369.660398,RH-2,True,MULTIPOLYGON (((-122.46116010700887 37.7654303...,RH-2
6067,0125049,RH3,930,0.0,129640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
combined_assessor.loc[combined_assessor.zoning_x=='PDR2']

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y
138477,3821009,PDR2,0,13358.00,39989,92464.0,92458.0,3821009,0.0,0.0,VACANT,0.0,13381.795069,PDR-1-D,True,MULTIPOLYGON (((-122.39876573712424 37.7664213...,PDR-1-D
146248,4288004,PDR2,31546,52200.00,3802482,94166.0,94157.0,4288004,31104.0,6292.0,MIXED,3368.0,49446.269364,PDR-2,True,MULTIPOLYGON (((-122.39348424412417 37.7506985...,PDR-2
146249,4288005,PDR2,64000,81237.00,8175338,94167.0,94158.0,4288005,64000.0,19932.0,PDR,17932.0,85616.458300,PDR-2,True,MULTIPOLYGON (((-122.39354920559127 37.7500063...,PDR-2
146250,4288006,PDR2,43200,64800.00,3591233,94168.0,94159.0,4288006,43200.0,55300.0,MIXED,10914.0,63526.786658,PDR-2,True,MULTIPOLYGON (((-122.39462070874185 37.7499444...,PDR-2
146253,4290011,PDR2,9300,9361.00,838063,94327.0,94318.0,4290011,9300.0,4443.0,MIXED,0.0,9578.749982,PDR-2,True,MULTIPOLYGON (((-122.39259612029906 37.7504734...,PDR-2
146630,4343001B,PDR2,35114,69996.00,7767079,103486.0,103464.0,4343001B,35114.0,5168.0,PDR,5168.0,69618.193281,PDR-2,True,MULTIPOLYGON (((-122.397193308955 37.749182371...,PDR-2
146631,4343001D,PDR2,19232,133297.00,3728545,103487.0,103465.0,4343001D,19232.0,2246.0,RETAIL/ENT,0.0,128261.904609,PDR-2,True,MULTIPOLYGON (((-122.40166401819543 37.7470196...,PDR-2
146646,4343030,PDR2,6834,17484.00,929495,94343.0,94334.0,4343030,6834.0,0.0,PDR,0.0,19108.892097,PDR-2,True,MULTIPOLYGON (((-122.40169703643947 37.7470185...,PDR-2
146650,4346003,PDR2,74122,343882.00,34173315,94192.0,94183.0,4346003,74122.0,0.0,PDR,0.0,343542.118835,PDR-2,True,MULTIPOLYGON (((-122.39372432462824 37.7496763...,PDR-2
146653,NaN,PDR2,108793,181269.00,20415781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#make new zoning column as a string data type based on existing zoning_x column

In [29]:
combined_assessor['finzone'] = combined_assessor['zoning_x'].astype(str)

In [30]:
combined_assessor.head()

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y,finzone
0,0001001,P,0,27965.0,0,147396.0,147394.0,0001001,0.0,0.0,PDR,0.0,28268.231998,P,True,MULTIPOLYGON (((-122.4210628988343 37.80860124...,P,P
1,0002001,C2,0,8359.0,0,72.0,73.0,0002001,0.0,0.0,PDR,0.0,8920.300030,C-2,True,MULTIPOLYGON (((-122.41981482898801 37.8087616...,C-2,C2
2,0004002,C2,0,0.0,0,6.0,7.0,0004002,0.0,20509.0,VACANT,0.0,31183.392188,C-2,True,MULTIPOLYGON (((-122.41379081959624 37.8085666...,C-2,C2
3,0005001,C2,0,68153.0,0,7.0,8.0,0005001,0.0,43725.0,PDR,0.0,69014.270707,C-2,True,MULTIPOLYGON (((-122.41593580428093 37.8082939...,C-2,C2
4,0006001,C2,0,108015.0,0,8.0,9.0,0006001,0.0,28728.0,PDR,0.0,106916.225315,C-2,True,MULTIPOLYGON (((-122.41758109128742 37.8080868...,C-2,C2


In [ ]:
#filter data based on zoning column to only include PDR districts

In [31]:
filteredPDR3 = combined_assessor[combined_assessor['finzone'].str.contains("PDR")]

In [32]:
filteredPDR3.sample(n=14)

,parcelno2,zoning_x,buildingsf,landarea,landvalue,Unnamed: 0,objectid,blklot,bldgsf,totuses,landuse,pdr,parcelarea,zoning_code,active,geometry,zoning_y,finzone
152267,5230034,PDR2,2659,0.0,348560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR2
148476,4786070,PDR2,2500,2500.0,344412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR2
140126,4019002,PDR1G,11824,6212.5,1904778,94317.0,94308.0,4019002,11824.0,7156.0,MIXED,4972.0,6146.260559,PDR-1-G,True,MULTIPOLYGON (((-122.4122370644569 37.76213473...,PDR-1-G,PDR1G
145079,4228025,PDR1G,1114,10000.0,422275,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR1G
159424,5585004,PDR2,17100,43769.0,792888,110173.0,110159.0,5585004,17100.0,19733.0,PDR,19733.0,43160.696272,PDR-2,True,MULTIPOLYGON (((-122.40185190182133 37.7400715...,PDR-2,PDR2
138602,3938001,PDR1D,103626,79997.0,21746655,85015.0,85001.0,3938001,103626.0,0.0,PDR,0.0,80427.742253,PDR-1-D,True,MULTIPOLYGON (((-122.40090324115202 37.7662908...,PDR-1-D,PDR1D
145090,4228036,PDR1G,1500,10000.0,346076,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR1G
118779,3574114,PDR1G,7913,3920.0,653663,82996.0,82979.0,3574114,7913.0,0.0,PDR,0.0,3879.684177,PDR-1-G,True,MULTIPOLYGON (((-122.41518349914048 37.7625565...,PDR-1-G,PDR1G
149340,4881027,PDR2,3796,0.0,134564,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PDR2
153842,5332026,PDR2,2880,3598.0,424415,90580.0,90577.0,5332026,2880.0,0.0,PDR,0.0,3577.372650,PDR-2,True,MULTIPOLYGON (((-122.40090303925706 37.7391760...,PDR-2,PDR2


In [33]:
filteredPDR3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 115261 to 161031
Data columns (total 18 columns):
parcelno2      178 non-null object
zoning_x       194 non-null object
buildingsf     194 non-null int64
landarea       194 non-null float64
landvalue      194 non-null int64
Unnamed: 0     139 non-null float64
objectid       139 non-null float64
blklot         139 non-null object
bldgsf         139 non-null float64
totuses        139 non-null float64
landuse        139 non-null object
pdr            139 non-null float64
parcelarea     139 non-null float64
zoning_code    139 non-null object
active         139 non-null object
geometry       139 non-null object
zoning_y       139 non-null object
finzone        194 non-null object
dtypes: float64(7), int64(2), object(9)
memory usage: 28.8+ KB


In [34]:
filteredPDR3.landarea.sum()

5130638.09

In [35]:
filteredPDR3.parcelarea.sum()

3979176.367905817

In [36]:
filteredPDR3.to_csv(r'filteredpdr3.csv')

In [ ]:
#playing around with which zoning column is most reliable to filter off of

In [37]:
combined_parcel['finzone2'] = combined_parcel['zoning_y'].astype(str)

In [38]:
filteredPDR4 = combined_parcel[combined_parcel['finzone2'].str.contains("PDR")]

In [39]:
filteredPDR4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139 entries, 69876 to 155456
Data columns (total 18 columns):
Unnamed: 0     139 non-null int64
objectid       139 non-null int64
blklot         139 non-null object
bldgsf         139 non-null int64
totuses        139 non-null int64
landuse        139 non-null object
pdr            139 non-null int64
parcelarea     139 non-null float64
zoning_code    139 non-null object
active         139 non-null object
geometry       139 non-null object
zoning_x       139 non-null object
parcelno2      139 non-null object
zoning_y       139 non-null object
buildingsf     139 non-null float64
landarea       139 non-null float64
landvalue      139 non-null float64
finzone2       139 non-null object
dtypes: float64(4), int64(5), object(9)
memory usage: 20.6+ KB


In [41]:
filteredpdrorig = pd.read_csv('filteredpdr.csv', low_memory=False)

In [42]:
filteredpdrorig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1448 entries, 0 to 1447
Data columns (total 12 columns):
Unnamed: 0     1448 non-null int64
objectid       1448 non-null int64
blklot         1448 non-null object
bldgsf         1448 non-null int64
totuses        1448 non-null int64
landuse        1448 non-null object
pdr            1448 non-null int64
parcelarea     1448 non-null float64
zoning_code    1448 non-null object
active         1448 non-null bool
geometry       1448 non-null object
zoning         1448 non-null object
dtypes: bool(1), float64(1), int64(5), object(5)
memory usage: 125.9+ KB
